# Introduction

In this notebook, I will attempt to tweak the Convolutional Neural Network model provided in the sample notebook, and report my findings. The list of tweaks are listed as follow (keeping everything else default):<br>

1) Fully connected layer output shape: (512, 256)(default) vs (256, 128) and (1024, 512)<br>
2) Dropout rate: 0.2(default) vs 0.1 vs 0.5<br>
3) 2 hiddenlayer vs 3 hiddenlayers: (512, 256)(default) vs [1024, 512, 256].<br>
4) learning rate: 0.1(default) vs 0.3 and 0.1/epochs(time-based decay)

## Load Python libraries

Let's load some useful Python libraries:
1. numpy: structural data types
2. pandas: data loading and manipulation
3. matplotlib, sns: data visualization
4. sklearn: data analytics algorithms
5. Tensorflow: Deep Learning library
6. Keras: Wrapper for Tensorflow

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K

from keras.datasets import cifar10


Using TensorFlow backend.


## Load data, perform reshape, normalization and one-hot encoding of labels

In [2]:
#number of final classes (in cifar10, there are 10 classes)
nb_classes = 10
nb_epoch = 100
batch_size = 128

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape(50000, 32 * 32 * 3)
x_test = x_test.reshape(10000, 32 * 32 * 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Define plot function:

In [3]:
def plot_train_acc(i, historyList):
    fig = plt.figure()
    for index, his in enumerate(historyList):
        plt.plot(range(nb_epoch),his.history['acc'],label='training'+str(index))
    plt.legend(loc=0)
    plt.xlabel('epochs')
    plt.ylabel('training_accuracy')
    plt.xlim([1,nb_epoch])
    plt.grid(True)
    plt.title("Training Accuracy Comparison")
    plt.show()
    fig.savefig('img/'+str(i)+'-training-accuracy.png')
    plt.close(fig)
    
def plot_val_acc(i, historyList):
    fig = plt.figure()
    for index, his in enumerate(historyList):
        plt.plot(range(nb_epoch),his.history['val_acc'],label='validation'+str(index))
    plt.legend(loc=0)
    plt.xlabel('epochs')
    plt.ylabel('validation_accuracy')
    plt.xlim([1,nb_epoch])
    plt.grid(True)
    plt.title("Validation Accuracy Comparison")
    plt.show()
    fig.savefig('img/'+str(i)+'-validation-accuracy.png')
    plt.close(fig)
    
def saveHistory(history, filename):
    import json
    json.dump(history.history, open('json_history/'+filename+'.json', 'w+'))

## 1) Different Output Shape:

Output shape refers to the number of neurons in each layer. In the sample, there are 4 layers:
Input Layer, 2 hidden layers with output shape: 512 and 256 respectively, and an Output Layer with outputshape: 10

### Define base model function:

In [4]:
def CNN_shape(shape):

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(shape))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model



### Generate 3 different models with different output shapes and plot the graphs:

In [5]:
modelDef = CNN_shape(512) #default model
model1 = CNN_shape(256)
model2 = CNN_shape(1024)

historyDef = modelDef.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(historyDef,'historyDef_CNN')

history1 = model1.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history1,'history1_CNN')

history2 = model2.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history2,'history2_CNN')

plot_train_acc(1, [historyDef, history1, history2])
plot_val_acc(2, [historyDef, history1, history2])


ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=2

## 2) Different dropout rate:

### Define base model function:

In [ ]:
def CNN_drop_rate(drop_rate1, drop_rate2):

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(drop_rate1))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(drop_rate2))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

### Generate 3 different models with different drop rate and plot the graphs:

In [ ]:
model3 = MLP_drop_rate(0.1,0.3)
model4 = MLP_drop_rate(0.5,0.5)

history3 = model3.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history3,'history3_CNN')
history4 = model4.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history4,'history4_CNN')

plot_train_acc(3, [historyDef, history3, history4])
plot_val_acc(4, [historyDef, history3, history4])

## 3) Different number of layers:

In [ ]:
# model 5 has 3 hiden layers
model5 = Sequential()
model5.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model5.add(Activation('relu'))
model5.add(Conv2D(32,(3, 3)))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Dropout(0.25))

model5.add(Flatten())
model5.add(Dense(512))
model5.add(Activation('relu'))
model5.add(Dropout(0.5))
model5.add(Dense(256))
model5.add(Activation('relu'))
model5.add(Dropout(0.5))
model5.add(Dense(nb_classes))
model5.add(Activation('softmax'))

model5.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

history5 = model5.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history5,'history5')
           
plot_train_acc(5 [historyDef, history5])
plot_val_acc(6, [historyDef, history5])

## 4) Different Learning Rate

### Define base model function:

In [ ]:
def MLP_learningRate(lr, decay):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=lr, momentum=0.0, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    #model.summary()
    return(model)

In [ ]:
model6 = MLP_learningRate(0.3, 0.0)
model7 = MLP_learningRate(0.1, 0.1/nb_epoch)

history6 = model6.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history6,'history6')
history7 = model7.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history7,'history7')

plot_train_acc(7, [historyDef, history6, history7])
plot_val_acc(8, [historyDef, history6, history7])

## 5) Different Activation Functions:

### Define base model function:

In [ ]:
def MLP_act_func(act_func):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation(act_func))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation(act_func))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    #model.summary()
    return(model)

In [ ]:
model8 = MLP_act_func('sigmoid')
model9 = MLP_act_func('prelu')
model10 = MLP_act_func('softplus')

history8 = model8.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history8,'history8')
history9 = model9.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history9,'history9')
history10 = model10.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history10,'history10')

plot_train_acc(7, [historyDef, history8, history9, history10])
plot_val_acc(8, [historyDef, history8, history9, history10])

## 6) Different Loss Function:
Since there are 10 classes in this dataset, 'categorical cross entropy' is the only option

In [ ]:
def MLP_loss_func(loss_func):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss=loss_func,
                  optimizer='sgd',
                  metrics=['accuracy'])
    #model.summary()
    return(model)

In [ ]:
model11 = MLP_loss_func('sparse_categorical_crossentropy')


history11 = model11.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history11,'history11')

plot_train_acc(10, [historyDef, history11])
plot_val_acc(12, [historyDef, history11])

You can add your analysis and results using these markdown cells.
Some of the things you can play with for your models include
1. Effects of varying number of hidden layers
2. Effects of varying number of neurons in the hidden layers
3. Effects of different learning rates
4. Effects of other parameters (training method, activation function, loss function etc)

You can also change the metric for classifier performance by using other measures such as mean squared error, confusion matrices etc.

Here are some useful links for learning more about creating NN architectures:
1. Keras - http://keras.io
2. Tensorflow - http://www.tensorflow.org/
3. Python numpy tutorial -  http://cs231n.github.io/python-numpy-tutorial/
